# Model Layers

This module contains many layer classes that we might be interested in using in our models. These layers complement the default [Pytorch layers](https://pytorch.org/docs/stable/nn.html) which we can also use as predefined layers.

In [ ]:
from fastai import *
from fastai.vision import *
from fastai.gen_doc.nbdoc import *

In [ ]:
show_doc(AdaptiveConcatPool2d, doc_string=False)

<h2 id="AdaptiveConcatPool2d"><code>class</code> <code>AdaptiveConcatPool2d</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L55" class="source_link">[source]</a></h2>

> <code>AdaptiveConcatPool2d</code>(`sz`:`Optional`\[`int`\]=`None`) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.layers import * 

Layer that concats `AdaptiveAvgPool2d` and `AdaptiveMaxPool2d`. Output will be `2*sz` or 2 if `sz` is None.

The [`AdaptiveConcatPool2d`](/layers.html#AdaptiveConcatPool2d) object uses adaptive average pooling and adaptive max pooling and concatenates them both. We use this because it provides the model with the information of both methods and improves performance. This technique is called `adaptive` because it allows us to decide on what output dimensions we want, instead of choosing the input's dimensions to fit a desired output size.

Let's try training with Adaptive Average Pooling first, then with Adaptive Max Pooling and finally with the concatenation of them both to see how they fare in performance.

We will first define a [`simple_cnn`](/layers.html#simple_cnn) using [Adapative Max Pooling](https://pytorch.org/docs/stable/nn.html#torch.nn.AdaptiveMaxPool2d) by changing the source code a bit.

In [ ]:
path = untar_data(URLs.MNIST_SAMPLE)
data = ImageDataBunch.from_folder(path)

In [ ]:
def simple_cnn_max(actns:Collection[int], kernel_szs:Collection[int]=None,
               strides:Collection[int]=None) -> nn.Sequential:
    "CNN with `conv2d_relu` layers defined by `actns`, `kernel_szs` and `strides`"
    nl = len(actns)-1
    kernel_szs = ifnone(kernel_szs, [3]*nl)
    strides    = ifnone(strides   , [2]*nl)
    layers = [conv_layer(actns[i], actns[i+1], kernel_szs[i], stride=strides[i])
        for i in range(len(strides))]
    layers.append(nn.Sequential(nn.AdaptiveMaxPool2d(1), Flatten()))
    return nn.Sequential(*layers)

In [ ]:
model = simple_cnn_max((3,16,16,2))
learner = Learner(data, model, metrics=[accuracy])
learner.fit(1)

Total time: 00:04
epoch  train_loss  valid_loss  accuracy
1      0.107965    0.073804    0.987733  (00:04)



Now let's try with [Adapative Average Pooling](https://pytorch.org/docs/stable/nn.html#torch.nn.AdaptiveAvgPool2d) now.

In [ ]:
def simple_cnn_avg(actns:Collection[int], kernel_szs:Collection[int]=None,
               strides:Collection[int]=None) -> nn.Sequential:
    "CNN with `conv2d_relu` layers defined by `actns`, `kernel_szs` and `strides`"
    nl = len(actns)-1
    kernel_szs = ifnone(kernel_szs, [3]*nl)
    strides    = ifnone(strides   , [2]*nl)
    layers = [conv_layer(actns[i], actns[i+1], kernel_szs[i], stride=strides[i])
        for i in range(len(strides))]
    layers.append(nn.Sequential(nn.AdaptiveAvgPool2d(1), Flatten()))
    return nn.Sequential(*layers)

In [ ]:
model = simple_cnn_avg((3,16,16,2))
learner = Learner(data, model, metrics=[accuracy])
learner.fit(1)

Total time: 00:03
epoch  train_loss  valid_loss  accuracy
1      0.253523    0.204772    0.975466  (00:03)



Finally we will try with the concatenation of them both [`AdaptiveConcatPool2d`](/layers.html#AdaptiveConcatPool2d). We will see that, in fact, it increases our accuracy and decreases our loss considerably!

In [ ]:
def simple_cnn(actns:Collection[int], kernel_szs:Collection[int]=None,
               strides:Collection[int]=None) -> nn.Sequential:
    "CNN with `conv2d_relu` layers defined by `actns`, `kernel_szs` and `strides`"
    nl = len(actns)-1
    kernel_szs = ifnone(kernel_szs, [3]*nl)
    strides    = ifnone(strides   , [2]*nl)
    layers = [conv_layer(actns[i], actns[i+1], kernel_szs[i], stride=strides[i])
        for i in range(len(strides))]
    layers.append(nn.Sequential(AdaptiveConcatPool2d(1), Flatten()))
    return nn.Sequential(*layers)

In [ ]:
model = simple_cnn((3,16,16,2))
learner = Learner(data, model, metrics=[accuracy])
learner.fit(1)

Total time: 00:03
epoch  train_loss  valid_loss  accuracy
1      0.107713    0.068680    0.994603  (00:03)



In [ ]:
show_doc(Lambda, doc_string=False)

<h2 id="Lambda"><code>class</code> <code>Lambda</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L8" class="source_link">[source]</a></h2>

> <code>Lambda</code>(`func`:`LambdaFunc`) :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Lambda allows us to define functions and use them as layers in our networks inside a [Sequential](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential) object. 

So, for example, say we want to apply a [log_softmax loss](https://pytorch.org/docs/stable/nn.html#torch.nn.functional.log_softmax) and we need to change the shape of our output batches to be able to use this loss. We can add a layer that applies the necessary change in shape by calling:

`Lambda(lambda x: x.view(x.size(0),-1))`

Let's see an example of how the shape of our output can change when we add this layer.

In [ ]:
model = nn.Sequential(
    nn.Conv2d(3,  16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
)

model.cuda()

for xb, yb in data.train_dl:
    out = (model(*[xb]))
    print(out.size())
    break

torch.Size([64, 10, 1, 1])


In [ ]:
model = nn.Sequential(
    nn.Conv2d(3,  16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
    Lambda(lambda x: x.view(x.size(0),-1))
)

model.cuda()

for xb, yb in data.train_dl:
    out = (model(*[xb]))
    print(out.size())
    break

torch.Size([64, 10])


In [ ]:
show_doc(Flatten)

<h4 id="Flatten"><code>Flatten</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L21" class="source_link">[source]</a></h4>

> <code>Flatten</code>() → `Tensor`

Flattens `x` to a single dimension, often used at the end of a model.  

The function we build above is actually implemented in our library as [`Flatten`](/layers.html#Flatten). We can see that it returns the same size when we run it.

In [ ]:
model = nn.Sequential(
    nn.Conv2d(3,  16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
    Flatten(),
)

model.cuda()

for xb, yb in data.train_dl:
    out = (model(*[xb]))
    print(out.size())
    break

torch.Size([64, 10])


In [ ]:
show_doc(PoolFlatten)

<h4 id="PoolFlatten"><code>PoolFlatten</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L25" class="source_link">[source]</a></h4>

> <code>PoolFlatten</code>() → [`Sequential`](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential)

Apply [`nn.AdaptiveAvgPool2d`](https://pytorch.org/docs/stable/nn.html#torch.nn.AdaptiveAvgPool2d) to `x` and then flatten the result.  

We can combine these two final layers ([AdaptiveAvgPool2d](https://pytorch.org/docs/stable/nn.html#torch.nn.AdaptiveAvgPool2d) and [`Flatten`](/layers.html#Flatten)) by using [`PoolFlatten`](/layers.html#PoolFlatten).

In [ ]:
model = nn.Sequential(
    nn.Conv2d(3,  16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    PoolFlatten()
)

model.cuda()

for xb, yb in data.train_dl:
    out = (model(*[xb]))
    print(out.size())
    break

torch.Size([64, 10])


In [ ]:
show_doc(ResizeBatch)

<h4 id="ResizeBatch"><code>ResizeBatch</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L17" class="source_link">[source]</a></h4>

> <code>ResizeBatch</code>(`size`:`int`) → `Tensor`

Layer that resizes x to `size`, good for connecting mismatched layers.  

Another use we give to the Lambda function is to resize batches with [`ResizeBatch`](/layers.html#ResizeBatch) when we have a layer that expects a different input than what comes from the previous one. Let's see an example:

In [ ]:
a = torch.tensor([[1., -1.], [1., -1.]])
print(a)

tensor([[ 1., -1.],
        [ 1., -1.]])


In [ ]:
out = ResizeBatch(4)
print(out(a))

tensor([[ 1., -1.,  1., -1.]])


In [ ]:
show_doc(CrossEntropyFlat, doc_string=False)

<h2 id="CrossEntropyFlat"><code>class</code> <code>CrossEntropyFlat</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L97" class="source_link">[source]</a></h2>

> <code>CrossEntropyFlat</code>(`weight`=`None`, `size_average`=`None`, `ignore_index`=`-100`, `reduce`=`None`, `reduction`=`'elementwise_mean'`) :: [`CrossEntropyLoss`](https://pytorch.org/docs/stable/nn.html#torch.nn.CrossEntropyLoss)

Same as [nn.CrossEntropyLoss](https://pytorch.org/docs/stable/nn.html#torch.nn.CrossEntropyLoss), but flattens input and target. Is used to calculate cross entropy on arrays (which Pytorch will not let us do with their [nn.CrossEntropyLoss](https://pytorch.org/docs/stable/nn.html#torch.nn.CrossEntropyLoss) function). An example of a use case is image segmentation models where the output in an image (or an array of pixels).

The parameters are the same as [nn.CrossEntropyLoss](https://pytorch.org/docs/stable/nn.html#torch.nn.CrossEntropyLoss): `weight` to rescale each class, `size_average` whether we want to sum the losses across elements in a batch or we want to add them up, `ignore_index` what targets do we want to ignore, `reduce` on whether we want to return a loss per batch element and `reduction` specifies which type of reduction (if any) we want to apply to our input.

In [ ]:
show_doc(MSELossFlat)

<h2 id="MSELossFlat"><code>class</code> <code>MSELossFlat</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L103" class="source_link">[source]</a></h2>

> <code>MSELossFlat</code>(`size_average`=`None`, `reduce`=`None`, `reduction`=`'elementwise_mean'`) :: [`MSELoss`](https://pytorch.org/docs/stable/nn.html#torch.nn.MSELoss)

Same as [`nn.MSELoss`](https://pytorch.org/docs/stable/nn.html#torch.nn.MSELoss), but flattens input and target.  

In [ ]:
show_doc(Debugger)

<h2 id="Debugger"><code>class</code> <code>Debugger</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L64" class="source_link">[source]</a></h2>

> <code>Debugger</code>() :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

A module to debug inside a model.  

The debugger module allows us to peek inside a network while its training and see in detail what is going on. We can see inputs, ouputs and sizes at any point in the network.

For instance, if you run the following:

``` python
model = nn.Sequential(
    nn.Conv2d(3,  16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    Debugger(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1), nn.ReLU(),
)

model.cuda()

learner = Learner(data, model, metrics=[accuracy])
learner.fit(5)
```
... you'll see something like this:

```
/home/ubuntu/fastai/fastai/layers.py(74)forward()
     72     def forward(self,x:Tensor) -> Tensor:
     73         set_trace()
---> 74         return x
     75 
     76 class StdUpsample(nn.Module):

ipdb>
```

In [ ]:
show_doc(NoopLoss)

<h2 id="NoopLoss"><code>class</code> <code>NoopLoss</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L108" class="source_link">[source]</a></h2>

> <code>NoopLoss</code>() :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Just returns the `output`.  

In [ ]:
show_doc(PixelShuffle_ICNR)

<h2 id="PixelShuffle_ICNR"><code>class</code> <code>PixelShuffle_ICNR</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L88" class="source_link">[source]</a></h2>

> <code>PixelShuffle_ICNR</code>(`ni`:`int`, `nf`:`int`=`None`, `scale`:`int`=`2`) :: [`Sequential`](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential)

Upsample by `scale` from `ni` filters to `nf` (default `ni`), using [`nn.PixelShuffle`](https://pytorch.org/docs/stable/nn.html#torch.nn.PixelShuffle), [`icnr`](/layers.html#icnr) init, and [`weight_norm`](https://pytorch.org/docs/stable/nn.html#torch.nn.weight_norm).  

In [ ]:
show_doc(bn_drop_lin, doc_string=False)

<h4 id="bn_drop_lin"><code>bn_drop_lin</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L29" class="source_link">[source]</a></h4>

> <code>bn_drop_lin</code>(`n_in`:`int`, `n_out`:`int`, `bn`:`bool`=`True`, `p`:`float`=`0.0`, `actn`:`Optional`\[[`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)\]=`None`)

The [`bn_drop_lin`](/layers.html#bn_drop_lin) function returns a sequence of [batch normalization](https://arxiv.org/abs/1502.03167), [dropout](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf) and a linear layer. This custom layer is usually used at the end of a model. 

`n_in` represents the number of size of the input `n_out` the size of the output, `bn` whether we want batch norm or not, `p` is how much dropout and `actn` is an optional parameter to add an activation function at the end.

In [ ]:
show_doc(conv2d)

<h4 id="conv2d"><code>conv2d</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L37" class="source_link">[source]</a></h4>

> <code>conv2d</code>(`ni`:`int`, `nf`:`int`, `ks`:`int`=`3`, `stride`:`int`=`1`, `padding`:`int`=`None`, `bias`=`False`) → [`Conv2d`](https://pytorch.org/docs/stable/nn.html#torch.nn.Conv2d)

Create [`nn.Conv2d`](https://pytorch.org/docs/stable/nn.html#torch.nn.Conv2d) layer: `ni` inputs, `nf` outputs, `ks` kernel size. `padding` defaults to `k//2`.  

In [ ]:
show_doc(conv2d_trans)

<h4 id="conv2d_trans"><code>conv2d_trans</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L42" class="source_link">[source]</a></h4>

> <code>conv2d_trans</code>(`ni`:`int`, `nf`:`int`, `ks`:`int`=`2`, `stride`:`int`=`2`, `padding`:`int`=`0`, `bias`=`False`) → [`ConvTranspose2d`](https://pytorch.org/docs/stable/nn.html#torch.nn.ConvTranspose2d)

Create [`nn.ConvTranspose2d`](https://pytorch.org/docs/stable/nn.html#torch.nn.ConvTranspose2d) layer: `ni` inputs, `nf` outputs, `ks` kernel size, `stride`: stride. `padding` defaults to 0.  

In [ ]:
show_doc(conv_layer, doc_string=False)

<h4 id="conv_layer"><code>conv_layer</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L46" class="source_link">[source]</a></h4>

> <code>conv_layer</code>(`ni`:`int`, `nf`:`int`, `ks`:`int`=`3`, `stride`:`int`=`1`, `padding`:`int`=`None`, `bias`:`bool`=`False`, `bn`:`bool`=`True`, `leaky`:`float`=`None`, `transpose`:`bool`=`False`)

The [`conv_layer`](/layers.html#conv_layer) function returns a sequence of [nn.Conv2D](https://pytorch.org/docs/stable/nn.html#torch.nn.Conv2d), [BatchNorm](https://arxiv.org/abs/1502.03167) and a ReLU or [leaky RELU](https://ai.stanford.edu/~amaas/papers/relu_hybrid_icml2013_final.pdf) activation function.

`n_in` represents the number of size of the input `n_out` the size of the output, `ks` kernel size, `stride` the stride with which we want to apply the convolutions. `bias` will decide if they have bias or not, we only put a `BachNorm` layer if `bn=True`. If `leaky` is None, the activation is a standard `ReLU`, otherwise it's a `LearkyReLU` of slope `leaky`. Finally if `transpose=True`, the convolution is replaced by a `ConvTranspose2D`.

In [ ]:
show_doc(embedding, doc_string=False)

<h4 id="embedding"><code>embedding</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L128" class="source_link">[source]</a></h4>

> <code>embedding</code>(`ni`:`int`, `nf`:`int`) → [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Create an [embedding layer](https://arxiv.org/abs/1711.09160) with input size `ni` and output size `nf`.

In [ ]:
show_doc(simple_cnn)

<h4 id="simple_cnn"><code>simple_cnn</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L112" class="source_link">[source]</a></h4>

> <code>simple_cnn</code>(`actns`:`Collection`\[`int`\], `kernel_szs`:`Collection`\[`int`\]=`None`, `strides`:`Collection`\[`int`\]=`None`, `bn`=`False`) → [`Sequential`](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential)

CNN with [`conv_layer`](/layers.html#conv_layer) defined by `actns`, `kernel_szs` and `strides`, plus batchnorm if `bn`.  

In [ ]:
show_doc(std_upsample_head, doc_string=False)

<h4 id="std_upsample_head"><code>std_upsample_head</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L70" class="source_link">[source]</a></h4>

> <code>std_upsample_head</code>(`c`, `nfs`:`Collection`\[`int`\]) → [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Create a sequence of upsample layers with a RELU at the beggining and a [nn.ConvTranspose2d](https://pytorch.org/docs/stable/nn.html#torch.nn.ConvTranspose2d). 

`nfs` is a list with the input and output sizes of each upsample layer and `c` is the output size of the final 2D Transpose Convolutional layer.

In [ ]:
show_doc(trunc_normal_)

<h4 id="trunc_normal_"><code>trunc_normal_</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L123" class="source_link">[source]</a></h4>

> <code>trunc_normal_</code>(`x`:`Tensor`, `mean`:`float`=`0.0`, `std`:`float`=`1.0`) → `Tensor`

Truncated normal initialization.  

In [ ]:
show_doc(icnr)

<h4 id="icnr"><code>icnr</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L78" class="source_link">[source]</a></h4>

> <code>icnr</code>(`x`, `scale`=`2`, `init`=`'kaiming_normal_'`)

ICNR init.  

## Undocumented Methods - Methods moved below this line will intentionally be hidden

In [ ]:
show_doc(Debugger.forward)

<h4 id="Debugger.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L66" class="source_link">[source]</a></h4>

> <code>forward</code>(`x`:`Tensor`) → `Tensor`

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(MSELossFlat.forward)

<h4 id="MSELossFlat.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L105" class="source_link">[source]</a></h4>

> <code>forward</code>(`input`:`Tensor`, `target`:`Tensor`) → `Rank0Tensor`

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(CrossEntropyFlat.forward)

<h4 id="CrossEntropyFlat.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L99" class="source_link">[source]</a></h4>

> <code>forward</code>(`input`:`Tensor`, `target`:`Tensor`) → `Rank0Tensor`

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(Lambda.forward)

<h4 id="Lambda.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L15" class="source_link">[source]</a></h4>

> <code>forward</code>(`x`)

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(AdaptiveConcatPool2d.forward)

<h4 id="AdaptiveConcatPool2d.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L62" class="source_link">[source]</a></h4>

> <code>forward</code>(`x`)

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(NoopLoss.forward)

<h4 id="NoopLoss.forward"><code>forward</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L110" class="source_link">[source]</a></h4>

> <code>forward</code>(`output`, `target`)

Defines the computation performed at every call. Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them. 

In [ ]:
show_doc(NoopLoss)

<h2 id="NoopLoss"><code>class</code> <code>NoopLoss</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L108" class="source_link">[source]</a></h2>

> <code>NoopLoss</code>() :: [`Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module)

Just returns the `output`.  

In [ ]:
show_doc(PixelShuffle_ICNR)

<h2 id="PixelShuffle_ICNR"><code>class</code> <code>PixelShuffle_ICNR</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L88" class="source_link">[source]</a></h2>

> <code>PixelShuffle_ICNR</code>(`ni`:`int`, `nf`:`int`=`None`, `scale`:`int`=`2`) :: [`Sequential`](https://pytorch.org/docs/stable/nn.html#torch.nn.Sequential)

Upsample by `scale` from `ni` filters to `nf` (default `ni`), using [`nn.PixelShuffle`](https://pytorch.org/docs/stable/nn.html#torch.nn.PixelShuffle), [`icnr`](/layers.html#icnr) init, and [`weight_norm`](https://pytorch.org/docs/stable/nn.html#torch.nn.weight_norm).  

In [ ]:
show_doc(icnr)

<h4 id="icnr"><code>icnr</code><a href="https://github.com/fastai/fastai/blob/master/fastai/layers.py#L78" class="source_link">[source]</a></h4>

> <code>icnr</code>(`x`, `scale`=`2`, `init`=`'kaiming_normal_'`)

ICNR init.  

## New Methods - Please document or move to the undocumented section